# Objective: Issue Verification Codes to patients stored as REDCap records

Case Investigation and Contact Tracing (CICT) migration to REDCap will require an issue of Exposure Notification codes to patients that
have provided a phone number in the REDCap records.

#### Meeting with Daniel L. 02/23/2022
 - server may issue 500 code during server maintenance, response may return as text instead of json. Implement retry logic with exponential back-off algorithm. 
 - 400 errors, bad phone number, or date out of range, code will not be issued. Need logic to record case outcome (in redcap?)
 - padding, create a function to randomly generate string of length 1-2624, input as padding. 
 - generate UUID (UNiversal Unique Identifier):
     - use MD5 function (pre-made in python, numpy?)
     - two arguments as input to MD5: patient phone number, unique argument to me (I generate this). 
     - Out put of MD5 is my UUID, include that to send to server. 
     - server will store UUID, this will be used to prevent repeat code issuing in the future. 

#### Questions:
    - difference between verification code and long term verification token?
    - what is verification certificate?

In [1]:
import requests
import json
import string
import random
import hashlib

from config import api_key
from config import uuid_salt

# Single code issue example:

In [2]:
hed = {
    'x-api-key': api_key,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
    }
url = "https://adminapi.encv-test.org/api/issue"

data = json.dumps({
    "symptomDate": "2022-02-06",
    "testDate": "2022-02-07",
    "testType": "confirmed",
    "tzOffset": -480,
    "phone": "+1 971-222-7382",
    "padding": "U11krYIjr8eX"
    })

In [3]:
response = requests.post(url, headers=hed, data=data)
print(response.status_code)

200


In [4]:
print(response.request.headers)
print(response.request.body)
print(response.request.url)

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': 'application/json', 'Connection': 'keep-alive', 'x-api-key': 'IoFQd8VXQ5dZ6ELB9t1W7NasfjpZSF8q1HxkKecp6lNfjvgMt2HQcUz6C9wnmVNzhBZwRKk8LVR13dr3U4jaKg.4.737tohTODPGj_ctg1Mt6a3HjZnW_co5loVx1AZdHBfm1wh9W7WIS6VDmK-u0bhzYgURfL-NEWXJ1Z23oA7EDnQ', 'Content-Type': 'application/json', 'Content-Length': '153'}
{"symptomDate": "2022-02-06", "testDate": "2022-02-07", "testType": "confirmed", "tzOffset": -480, "phone": "+1 971-222-7382", "padding": "U11krYIjr8eX"}
https://adminapi.encv-test.org/api/issue


In [5]:
response.text

'{"padding":"Cz/XbHunm/vu9z7cWzD+GNpP9LBIbFXqtHu/S948V21FFeTNWyqzX2QJ1eH5QlJSaQH2GgkxugyHn13+KydQLdRVTG6oMvXGS1q/0RvFMBAEgNXTJb1T/Q20cW/tcx5vJmz1JIcSg5IyM59fwDz6D32q/d3wpkRU8azbdBa7eBlxriGMTwE6IOG1orZo59cW/68IoQdT4aKI14+54FW2iqYOnaviLM/aClhvaT/lT8kZig1D3K3kTNUP/uiLHwywyFOAGWaCpwH2Dw2hPyZ1H1UdesfMRXupJs1givBbUW5hTI9OzttV8w7U3VGmxsot2Kq0TQS9Pn6Bd1xCr2yA/izlfXngT/hwGMOxX9gTilvASCf+faX4BtPWqfuy/JPky6JPk7cHIi/cK0Sec04nftzB6VFhZOPBo3dUjZLbo2LywcSfWnYrAhiNm0YMhK/YZjcK4mJ49cvRWj9+h8COlVNM0bHZa6BOtL8plvkq8iUvhJxRD9cuRL/2LxOyqP4gUm5G/0mPrWD22ELNcIsD98yIbiZ2uRT4TmIzVA3DYotMjHFcjbxNS9LjHFZIZRvcit1ydcHUP2jVzFDcmLnxuw61MZtOhRzzi9mxsDq8UYPl7YFKtbKNlt1JgEckXf8F1CDzbB5FQxsxluTFS1m8Eog9458HKSPOOJYh0kCykwA7UWJ0MXRLF8/Rhdm9VOA4YtQ4GKftS2uRIDh4LQcYM9XpJDeiUFL/h+9wBGPjOczCehH5XWQeyfrpgTXGql/CaG3SmRh0i4ScxwJ6BlExElb7Sl2poNNAO/7gRNmrRJuk6y7SFQjcZ+GztUzx9bBcAoewK9MbG82gOFDs+tdZjZoEK9s03Ic3hyXNoj/6zBISoOaKBx/GI5VRUtB23+48yq7Df7RKXjJdBXmVl3yYvGYcdmtXsGFPuLgwE46ObvIfy9dei/XxVC92Oy88zN60wmG0a+o1MYUq3FVrOkmHS24wewt

# Batch Issue example

In [6]:
hed = {
    'x-api-key': api_key,
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

url = "https://adminapi.encv-test.org/api/batch-issue"

data = json.dumps({
    "codes": [
        {
            "symptomDate": "2022-02-06",
            "testDate": "2022-02-07",
            "testType": "confirmed",
            "tzOffset": -480,
            "phone": "+1 971-222-7382",
            
        },
        {
            "symptomDate": "2022-02-06",
            "testDate": "2022-02-07",
            "testType": "confirmed",
            "tzOffset": -480,
            "phone": "+1 971-222-7383",
        }
    ],
    "padding": "U11krYIjr8eX"
})

In [ ]:
response = requests.post(url, headers=hed, data=data)
print(response.status_code)

In [ ]:
response.request.body

In [ ]:
response.content

### Generate Padding Function

In [7]:
def padding_gen():
    """
    Generate a string of random length between 0 and 3000 characters
    to be used as padding for API calls
    
    return:
        string 
    """
    
    chars = string.ascii_uppercase + string.digits + string.ascii_lowercase
    length = random.randint(0,3000)
    
    return "".join([random.choice(chars) for i in range(length)])

### Generate UUID per phone number

In [10]:
def uuid_gen(text, salt = uuid_salt):
    """
    Generate a uuid from phone number string and HMAC key (unique passphrase)
    
    return:
        string
    """
    textutf8 = text.encode("utf-8")
    saltutf8 = salt.encode("utf-8")
    
    hash_key = hashlib.md5(saltutf8+b":"+textutf8)
    hexa = hash_key.hexdigest()
    
    return hexa
    